In [1]:
import os
from pypdf import PdfReader
import docx2txt
#from langchain.document_loaders import PyPDFLoader # alternativa nel reader

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.llms import HuggingFaceHub
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferWindowMemory

In [2]:
# extract text
text = docx2txt.process("./docs/Reg. persone fisiche Abbate-Desiderio.docx")

In [4]:
def read_file(filename):

    if ".pdf" in filename:
        document = ""
        reader = PdfReader(filename)
        for page in reader.pages:
            document += page.extract_text()
        
        # oppure anche
        # loader = PyPDFLoader(pdf_path)
        # return loader.load() # questo estra tutto il pdf
    
    elif ".docx" in filename:
        document = docx2txt.process(filename)

    return document

In [8]:
document_folder = "./docs/"
document_list = os.listdir(document_folder)
print(document_list)

['Reg. persone fisiche Abbate-Desiderio.docx', 'Reg. persone fisiche Ferrante-Iezzi.docx', 'Reg. persone fisiche Italiano-Pascarella.docx', 'Reg. persone fisiche Pedicini-Zorzi.docx', 'Registro cat. 117, imprese, gruppi di imprese e aziende.docx', 'Registro cat. Altro.docx', 'Registro cat. associazioni di categoria o di tutela di interessi diffusi.docx', 'Registro cat. associazioni di consumatori riconosciute.docx', 'Registro cat. associazioni professionali.docx', 'Registro cat. organizzazioni datoriali e sindacali.docx', 'Registro cat. organizzazioni non governative.docx', 'Registro cat. soggetti specializzati.docx']


In [9]:
docs = ""

for i, doc_name in enumerate(document_list):
    # leggo il singolo pdf
    doc = read_file(document_folder + doc_name)
    docs = docs + doc

In [10]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1200,
    chunk_overlap=300,
    separators=["\n\n\n\n","\n\n\n"]
)

# splitto tutti i documenti
splitted_docs = splitter.split_text(docs)

In [11]:
embedder = HuggingFaceEmbeddings(
    model_name="nickprock/sentence-bert-base-italian-uncased", # intfloat/multilingual-e5-large # sentence-transformers/all-MiniLM-L6-v2
    model_kwargs={'device': 'cpu'}
)

d:\anaconda3\envs\tm\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\anaconda3\envs\tm\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
"""
# carico tutti i documenti su FAISS
db_faiss = FAISS.from_texts(splitted_docs, embedder)
db_faiss.save_local('./faiss_db')
"""

In [12]:
db_faiss = FAISS.load_local('./faiss_db/', embedder, allow_dangerous_deserialization=True)

In [28]:
# creo il retriever dal db FAISS
retriever = db_faiss.as_retriever(search_kwargs={"k": 10})
retrieved_docs = retriever.invoke("di cosa si occupa Bloise Generoso?")

In [34]:
llm = HuggingFaceHub(
    repo_id = 'mistralai/Mixtral-8x7B-Instruct-v0.1', # tiiuae/falcon-7b-instruct
    #model_kwargs={"temperature": temperature, "max_length": max_length},
    huggingfacehub_api_token='your token here'
)

memory = ConversationBufferWindowMemory(
    k=0,
    memory_key="chat_history",
    output_key="answer",
    return_messages=True,
)

# Create the chatbot
qa_conversation = ConversationalRetrievalChain.from_llm(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    memory=memory,
)

In [35]:
question = "che cosa è la AIACE?"
response = qa_conversation({"question": question})

answer = response.get("answer").split("Helpful Answer:")[-1].strip()
print(answer)

AIACE è un'associazione che si occupa di tutelare e perseguire gli interessi dei consumatori in tema di sicurezza stradale, tutela dell'ambiente e tutela del credito. Si propone di formulare proposte, richieste, suggerimenti e promuovere studi e ricerche su tali tematiche.


In [36]:
question = ""
while "!!!" not in question:
    question = input()
    response = qa_conversation({"question": question})
    answer = response.get("answer").split("Helpful Answer:")[-1].strip()
    print(answer)

Yes, AIACE is involved in issues related to road safety.
AIACE è situata in Via Cesare Baronio n°146, 00176 Roma (RM).
Bloise Generoso è un rappresentante di interessi per se stesso e si occupa di favorire e incrementare la cooperazione fra i popoli e gli Stati europei; di favorire i processi di sviluppo, cooperazione e solidarietà internazionale; di concretizzare legami di amicizia, alleanza e comunione e fra i vari appartenenti ai servizi d’informazione, di tutela
Bloise Generoso è un rappresentante di interessi per se stesso e si occupa di favorire e incrementare la cooperazione fra i popoli e gli Stati europei; di favorire i processi di sviluppo, cooperazione e solidarietà internazionale; di concretizzare legami di amicizia, alleanza e comunione e fra i vari appartenenti ai servizi d’informazione, di tutela
The name of the person is not mentioned in the provided context.
Nuove Reti SRL
